data_loader에 대한 설명
https://dacon.io/codeshare/531?dtype=recent



In [ ]:
from sklearn.model_selection import KFold
import lightgbm

import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool 
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
from data_loader_v2 import data_loader_v2
import os
import pandas as pd
import numpy as np
import joblib

train_folder = '../0_Data/train/'
test_folder = '../0_Data/test/'
train_label_path = '../0_Data/train_label.csv'

#데이터 전처리

In [ ]:
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    
    return combined_df

In [ ]:
train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=10, nrows=60)
test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=20, nrows=60)

y = train['label']
train.drop('label',axis=1,inplace=True)

#모델 학습, 검증, 저장
Model Tuning & Evaluation

In [ ]:
parms = {
    'learning_rate' : 0.06,
    'num_leaves' : 400,
    'n_estimators' : 300,
    'max_depth': -1,
    'min_child_weight' : 3, 
    'subsample' : 0.8,
    'colsample_bytree' : 0.5,
    'objective' : 'multiclass',
    'n_jobs': -1
}

In [ ]:
# 4FOLD, 3SEED ENSEMBLE
# 총 12개의 모델을 평균내어 예측한다

lucky_seed=[4885,1992,1022]

for num,rs in enumerate(lucky_seed):

    kfold = KFold(n_splits=4, random_state = rs, shuffle = True)

    # dacon code
    cv=np.zeros((train.shape[0],198))

    for n, (train_idx, validation_idx) in enumerate(kfold.split(train)):

        x_train, x_validation = train.iloc[train_idx], train.iloc[validation_idx]
        y_train, y_validation = y.iloc[train_idx], y.iloc[validation_idx]

        model = lightgbm.LGBMClassifier(**parms, random_state=rs)

        model.fit(x_train, y_train, eval_set=[(x_validation, y_validation)], early_stopping_rounds= 30,
                  verbose=100) 
        joblib.dump(model, '../2_Code_pred/%s_fold_model_%s.pkl'%(n,rs))

        # CROSS-VALIDATION , EVALUATE CV
        cv[validation_idx,:] = model.predict_proba(x_validation)

In [ ]:
# MODEL LOAD & TEST PREDICT
# 12 MODELS 평균 사용
models = os.listdir('../2_Code_pred/')
models_list = [x for x in models if x.endswith(".pkl")]
assert len(models_list) ==12
temp_predictions = np.zeros((test.shape[0],198))

for m in models_list:
    model = joblib.load('../2_Code_pred/'+m)
    predict_proba = model.predict_proba(test)
    temp_predictions += predict_proba/12

In [ ]:
# dacon code
submission = pd.DataFrame(data=np.zeros((test.shape[0],198)))
submission.index = test.index 
submission.index.name = 'id'
submission+=temp_predictions

submission = submission.sort_index()
submission = submission.groupby('id').mean()
submission.to_csv('submission.csv', index=True)

#결과 및 결언
Conclusion & Discussion
데이터 전처리
PCA, Feature 정규화, Min-Max Scaling은 성능 향상에 도움이 되지 않음 Object와 NAN 값을 0으로 바꾸어 주는 전처리만 진행

모델 학습 검증
- Lgbm 모델 선택
Random Forest, Xgboost, LightGBM 모델 비교 결과 lgbm의 성능이 가장 좋았음

- K-fold & Random seed를 사용한 모델 하이퍼 파라미터 튜닝
Robust 한 모델을 만들기 위해 4Kfold * 3seed 총 12개의 모델을 만듬 Early stopping 값을 작게 설정하여 over-fitting 방지 min_child_weight 값을 CV를 통해 최적화 하여 over-fitting 방지 Soft-voting 예측 방법 선택

- Soft-voting 예측
예측 시 Hard-voting 방식과 Probability를 평균내는 Soft-voting 방식을 실험 evaluation metric이 log-loss였기 때문에 probability를 평균내는 방식의 성능이 좋았음 12개의 모델의 예측을 평균 하는 방식으로 최종 결과물 제출

데이터 처리 할 때

본 대회의 데이터는 칼럼이 많은 것이 특징입니다. 약 5000여개의 칼럼에서 row에 따라 값이 변하지 않는 칼럼은 의미없는 칼럼으로 간주하여 해당 칼럼을 삭제하는 작업을 진행하였습니다.

1) 유니크한 값의 개수가 1인 칼럼, 2) 표준편차가 0인 칼럼을 제거하는 방식을 적용하였습니다. 두 방법이 비슷해 보이지만, 1)의 경우 1783개, 2)의 경우 1693개의 칼럼이 삭제되어 약 100개 정도 차이가 있었습니다.

이 차이에 대해서는 스터디원 '인호베이션97'님 께서 토론 게시물을 작성해 주셨습니다. 링크를 참조해주세요.

https://dacon.io/competitions/official/235551/talkboard/401155?page=1&dtype=recent&ptype=pub

In [ ]:
#칼럼 drop ver1 : nunique()!=1
train_1 = train.loc[:,train.nunique()!=1]
train_1.shape

In [ ]:
#칼럼 drop ver2 : train.std()!=0
train_2 = train.loc[:,train.std()!=0]
train_2.shape

#데이터 불균형과 리샘플링
이번 문제의 경우 label이 197개가 존재하지만 각 label에 해당되는 개수가 다르기 때문에 학습이 제대로 진행되지 않을 수 있음(overfitting 가능성 높아짐 /
"예를 들어서 분포가 100개의 데이터에서 1과 0값이 각각 97 : 3 비율을 가지고 있을 때 모든 값을 1로 예측한다 하더라도 정확도가 97% 나오게 됩니다.")

# 이상치 탐색 및 제거
지금까지 칼럼 축소를 위주로 데이터 EDA를 진행하였는데, 이제 train 데이터셋의 row 데이터를 좀 더 살펴보겠습니다. 우리가 사용하고 있는 데이터 로드 모듈, data_loader_v2.py를 보면, 불러온 train 데이터의 row는 file id임을 알 수 있습니다.

따라서 row를 기준으로 이상치 탐색을 진행하면, 비교적 중요도가 낮은 file id를 제거할 수 있습니다. lsolation forest를 적용하여 이상치를 탐색을 진행했습니다. contamination변수를 0.01로 설정하여 전체 데이터 중 1%를 이상치로 간주하고 lf모델을 통해 이상치를 탐색한 결과입니다.


https://dacon.io/competitions/official/235551/codeshare/1377?page=1&dtype=recent&ptype=pub


하지만 이번 문제의 경우 이상치를 제거하지 않는 방법이 오히려 더 성능이 좋았다.
이상치가 오히려 특정 label을 나타내는 특징이 될 수 있기 때문
